In [1]:
import numpy as np
import time
x_train=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/dmnist20kfromsenet.npy')
x_train=x_train.reshape(20000,2000)
y=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/lmnist20kfromsenet.npy')
y=y.reshape(20000,)
alpha_trans1=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/cknnmnist20kfromsenet.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32') 
print(x_train.shape,y.shape,alpha_trans1.shape)

(20000, 2000) (20000,) (40, 500, 500)


In [2]:
!pip install memory_profiler
%load_ext memory_profiler

Defaulting to user installation because normal site-packages is not writeable


In [3]:
%memit

peak memory: 255.83 MiB, increment: 0.02 MiB


In [3]:
import glob
import numpy as np
import random
import tensorflow.compat.v1 as tf
import os
from sklearn.utils import shuffle
batchSize=500
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import fractional_matrix_power
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
from tensorflow.keras import backend as B
k_cluster = 10
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
from tensorflow.keras.models import Model
import sys
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from munkres import Munkres

from numpy import save


def eig_vec(x_train):
    nev_trans=[]
    SortedEigVec=np.zeros((batchSize,batchSize))
    for l in range (0,len(x_train),batchSize):
        xtr=x_train[l:l+batchSize]
        d=(euclidean_distances(xtr,xtr))
        W=np.zeros((len(xtr),len(xtr)))
        for i in range(len(xtr)):
            for j in range (len(xtr)):
                if i==j:
                    W[i,j]=0
                else:
                    W[i,j]=np.exp(-(d[i,j]**2)/(2*0.09))
        degree=np.diag(sum(W))
        L = degree-W
        eig_val,eig_vec=np.linalg.eig(L)
        dec_eigVal=np.flip(np.sort(eig_val))
        indexArr=(np.flip(np.argsort(eig_val)))
        for i in range (len(indexArr)):
            SortedEigVec[:,i]=eig_vec[:,indexArr[i]]
        SortedEigVec=np.array(SortedEigVec)
        nev_trans.append(SortedEigVec)
    return nev_trans,L,eig_val,eig_vec,dec_eigVal,W

def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    #print('uc1 shape is',l1)
    #print('uc2 shape is',l2)
    
    ## if 10==10 return true else return false
    assert(l1 == l2 and np.all(uc1 == uc2))
    m = np.ones([l1, l2])
    for i in range(l1):
        it_i = np.nonzero(c1 == uc1[i])[0]
        for j in range(l2):
            it_j = np.nonzero(c2 == uc2[j])[0]
            m_ij = np.intersect1d(it_j, it_i)
            m[i,j] =  -m_ij.size
    return m

def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes) 
    m = Munkres()
    indexes = m.compute(cost_matrix)
    mapper = { old: new for (old, new) in indexes }
    new_labels = translate_clustering(labels, mapper)
    new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
    return new_labels,accuracy(new_cm)


def arch():
    encoding_dim = 490 
    input_img = Input(shape=(2000,))
    encoded_layer_2 = Dense(1500, activation='tanh')(input_img)
    encoded_layer_3 = Dense(1000, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(700, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(700, activation='tanh')(encoded)
    decoded_layer_3 = Dense(1000, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(1500, activation='tanh')(decoded_layer_3)
    decoded = Dense(2000, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder

def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T      
    
    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    #print(q.shape)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + B.epsilon())).T
    return q

def asl():
    def myloss(y_true, y_pred):
        l1norm= l1norm_func(y_pred,y_true)
        l2norm= l2norm_func(y_pred,y_true)
        orthonormal= orthonormality_func(y_pred)
        structpreserve=structurePreserve_func(y_pred,alpha_trans1[g][:,:])
        return structpreserve+l1norm+l2norm+orthonormal
    return myloss

def l1norm_func(z,nt):
    q1=0.002*(tf.reduce_sum(tf.abs(tf.subtract(z,nt)),axis=1))
    return q1

def l2norm_func(z,nt):
    q2=0.02*(tf.sqrt(tf.reduce_sum(tf.square(z-nt))))
    return q2

def structurePreserve_func(z,ta):
    tz=tf.transpose(z)
    tz=tf.reshape(tz,(encoding_dim,batchSize))
    taa=tf.transpose(ta)
    taz=tf.matmul(tz,taa)
    taz=tf.reshape(taz,(encoding_dim,batchSize))
    q4=0.02*(tf.sqrt(tf.reduce_sum(tf.square(tz-taz))))
    q4=tf.transpose(q4)
    return q4

def orthonormality_func(z):
    z=[(z[i,:]/(tf.norm(z[i,:],2)))for i in range (batchSize)]
    transz=tf.transpose(z)
    q3=tf.tensordot(transz,z,axes=1)
    I=tf.eye(encoding_dim)
    q5=0.002*(tf.sqrt(tf.reduce_sum(tf.square(q3-I))))
    return q5



In [5]:
nev_trans,L,eig_val,eig_vec,dec_eigVal,W=eig_vec(x_train)

In [6]:
%memit

peak memory: 620.63 MiB, increment: 0.86 MiB


In [7]:
startae = time.time()
encoder,encoding_dim,autoencoder=arch()
autoencoder.compile(tf.keras.optimizers.Adam(lr=1e-3), loss=tf.keras.losses.MeanSquaredError())
autoencoder.fit(x_train,x_train,epochs=100,shuffle=True,batch_size=batchSize)
encoder.save_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/AE_mnist20k.ckpt')
endae=time.time()
print('AE training time is : ',endae - startae)

print('training from scratch')
encoder.load_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/AE_mnist20k.ckpt')
g=0
encoder.compile(tf.keras.optimizers.Adadelta(lr=1e-3, rho=0.95, decay=1e-4), loss=asl())
%memit


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 20000 samples
Epoch 1/100
20000/20000 [==============================] - 12s 594us/sample - loss: 3.7734e-04
Epoch 2/100
20000/20000 [==============================] - 12s 579us/sample - loss: 1.5202e-04
Epoch 3/100
20000/20000 [==============================] - 11s 564us/sample - loss: 1.1099e-04
Epoch 4/100
20000/20000 [==============================] - 11s 541us/sample - loss: 9.1226e-05
Epoch 5/100
20000/20000 [==============================] - 11s 549us/sample - loss: 7.9123e-05
Epoch 6/100
20000/20000 [==============================] - 10s 522us/sample - loss: 7.0721e-05
Epoch 7/100
20000/20000 [==============================] - 11s 540us/sample - loss: 6.4473e-05
Epoch 8/100
20000/20000 [==============================] - 10s 514us/sample - loss: 5.9619e-05
Epoch 9/100
20000/20000 [==============================] - 12s 586us/sample - loss: 5.5699e-05
Epoch 10/100
20000/20000 [===============

20000/20000 [==============================] - 8s 420us/sample - loss: 1.8282e-05
Epoch 83/100
20000/20000 [==============================] - 9s 435us/sample - loss: 1.8207e-05
Epoch 84/100
20000/20000 [==============================] - 8s 408us/sample - loss: 1.8103e-05
Epoch 85/100
20000/20000 [==============================] - 9s 430us/sample - loss: 1.8040e-05
Epoch 86/100
20000/20000 [==============================] - 9s 451us/sample - loss: 1.7965e-05
Epoch 87/100
20000/20000 [==============================] - 8s 421us/sample - loss: 1.7896e-05
Epoch 88/100
20000/20000 [==============================] - 8s 422us/sample - loss: 1.7811e-05
Epoch 89/100
20000/20000 [==============================] - 8s 422us/sample - loss: 1.7732e-05
Epoch 90/100
20000/20000 [==============================] - 8s 416us/sample - loss: 1.7652e-05
Epoch 91/100
20000/20000 [==============================] - 9s 451us/sample - loss: 1.7598e-05
Epoch 92/100
20000/20000 [==============================] - 9s 

In [8]:
start = time.time()
maxnum=0
for t in range (1,300,1):
    g=0
    for l in range (0,len(x_train),batchSize):
        encoder.fit(x_train[l:l+batchSize],nev_trans[g][:,batchSize-encoding_dim-1:batchSize-1],epochs=1,shuffle=False,batch_size=batchSize)
        g=g+1
    z_test=encoder.predict(x_train)
    k_means = KMeans(n_clusters=k_cluster, init='k-means++')
    ic= k_means.fit(z_test).cluster_centers_ 
    assignment = assign_func(z_test, ic)
    predictedlabels = np.argmax(assignment, axis=1)
    rectified_label, new_acc=rectify_label(predictedlabels, y)
    print('acc at',t, 'epoch is:',maxnum)
    if maxnum<new_acc:
        maxnum=new_acc
        encoder.save_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/sdsc_mnist20k.ckpt')
        np.save('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/mnistpca2000/sdsc_mnist20k.npy',ic)
        maxnmi=sklearn.metrics.normalized_mutual_info_score(y, predictedlabels)
        maxf1=sklearn.metrics.f1_score(y, rectified_label,average='micro')
        maxprec=sklearn.metrics.precision_score(y, rectified_label,average='weighted')
    else:
        continue
end=time.time()
print('E training time is : ',end - start)

Train on 500 samples
500/500 [==============================] - 14s 27ms/sample - loss: 1.4026
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.4153
Train on 500 samples
500/500 [==============================] - 0s 690us/sample - loss: 1.4156
Train on 500 samples
500/500 [==============================] - 0s 760us/sample - loss: 1.3914
Train on 500 samples
500/500 [==============================] - 0s 564us/sample - loss: 1.4084
Train on 500 samples
500/500 [==============================] - 0s 577us/sample - loss: 1.4072
Train on 500 samples
500/500 [==============================] - 0s 591us/sample - loss: 1.4037
Train on 500 samples
500/500 [==============================] - 0s 615us/sample - loss: 1.4046
Train on 500 samples
500/500 [==============================] - 0s 751us/sample - loss: 1.3996
Train on 500 samples
500/500 [==============================] - 0s 829us/sample - loss: 1.4188
Train on 500 samples
500/500 [======================

Train on 500 samples
500/500 [==============================] - 0s 602us/sample - loss: 1.3830
Train on 500 samples
500/500 [==============================] - 0s 695us/sample - loss: 1.3835
Train on 500 samples
500/500 [==============================] - 0s 612us/sample - loss: 1.3795
Train on 500 samples
500/500 [==============================] - 0s 662us/sample - loss: 1.3973
Train on 500 samples
500/500 [==============================] - 0s 527us/sample - loss: 1.3746
Train on 500 samples
500/500 [==============================] - 0s 579us/sample - loss: 1.3842
Train on 500 samples
500/500 [==============================] - 0s 552us/sample - loss: 1.3902
Train on 500 samples
500/500 [==============================] - 0s 636us/sample - loss: 1.3851
Train on 500 samples
500/500 [==============================] - 0s 713us/sample - loss: 1.3862
Train on 500 samples
500/500 [==============================] - 0s 611us/sample - loss: 1.3832
Train on 500 samples
500/500 [====================

Train on 500 samples
500/500 [==============================] - 0s 561us/sample - loss: 1.3723
Train on 500 samples
500/500 [==============================] - 0s 610us/sample - loss: 1.3675
Train on 500 samples
500/500 [==============================] - 0s 601us/sample - loss: 1.3686
Train on 500 samples
500/500 [==============================] - 0s 615us/sample - loss: 1.3650
Train on 500 samples
500/500 [==============================] - 0s 567us/sample - loss: 1.3638
Train on 500 samples
500/500 [==============================] - 0s 536us/sample - loss: 1.3667
Train on 500 samples
500/500 [==============================] - 0s 522us/sample - loss: 1.3711
Train on 500 samples
500/500 [==============================] - 0s 604us/sample - loss: 1.3681
Train on 500 samples
500/500 [==============================] - 0s 626us/sample - loss: 1.3682
Train on 500 samples
500/500 [==============================] - 0s 691us/sample - loss: 1.3632
Train on 500 samples
500/500 [====================

Train on 500 samples
500/500 [==============================] - 0s 584us/sample - loss: 1.3548
Train on 500 samples
500/500 [==============================] - 0s 679us/sample - loss: 1.3518
Train on 500 samples
500/500 [==============================] - 0s 736us/sample - loss: 1.3515
Train on 500 samples
500/500 [==============================] - 0s 650us/sample - loss: 1.3470
Train on 500 samples
500/500 [==============================] - 0s 626us/sample - loss: 1.3412
Train on 500 samples
500/500 [==============================] - 0s 657us/sample - loss: 1.3451
Train on 500 samples
500/500 [==============================] - 0s 770us/sample - loss: 1.3378
Train on 500 samples
500/500 [==============================] - 0s 748us/sample - loss: 1.3461
Train on 500 samples
500/500 [==============================] - 0s 528us/sample - loss: 1.3453
Train on 500 samples
500/500 [==============================] - 0s 576us/sample - loss: 1.3407
Train on 500 samples
500/500 [====================

Train on 500 samples
500/500 [==============================] - 0s 652us/sample - loss: 1.3229
Train on 500 samples
500/500 [==============================] - 0s 562us/sample - loss: 1.3314
Train on 500 samples
500/500 [==============================] - 0s 589us/sample - loss: 1.3296
Train on 500 samples
500/500 [==============================] - 0s 715us/sample - loss: 1.3259
Train on 500 samples
500/500 [==============================] - 0s 815us/sample - loss: 1.3367
Train on 500 samples
500/500 [==============================] - 0s 744us/sample - loss: 1.3461
Train on 500 samples
500/500 [==============================] - 0s 613us/sample - loss: 1.3318
Train on 500 samples
500/500 [==============================] - 0s 990us/sample - loss: 1.3256
Train on 500 samples
500/500 [==============================] - 0s 707us/sample - loss: 1.3344
Train on 500 samples
500/500 [==============================] - 0s 621us/sample - loss: 1.3348
Train on 500 samples
500/500 [====================

...

Train on 500 samples
500/500 [==============================] - 0s 744us/sample - loss: 1.0202
Train on 500 samples
500/500 [==============================] - 0s 955us/sample - loss: 1.0183
Train on 500 samples
500/500 [==============================] - 0s 853us/sample - loss: 1.0236
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0153
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0183
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0327
Train on 500 samples
500/500 [==============================] - 0s 897us/sample - loss: 1.0287
Train on 500 samples
500/500 [==============================] - 0s 992us/sample - loss: 1.0285
Train on 500 samples
500/500 [==============================] - 0s 915us/sample - loss: 1.0229
Train on 500 samples
500/500 [==============================] - 0s 837us/sample - loss: 1.0194
Train on 500 samples
500/500 [==========================

500/500 [==============================] - 0s 755us/sample - loss: 1.0280
Train on 500 samples
500/500 [==============================] - 0s 966us/sample - loss: 1.0278
Train on 500 samples
500/500 [==============================] - 0s 986us/sample - loss: 1.0223
Train on 500 samples
500/500 [==============================] - 0s 946us/sample - loss: 1.0188
Train on 500 samples
500/500 [==============================] - 0s 855us/sample - loss: 1.0358
Train on 500 samples
500/500 [==============================] - 0s 898us/sample - loss: 1.0286
Train on 500 samples
500/500 [==============================] - 0s 868us/sample - loss: 1.0254
Train on 500 samples
500/500 [==============================] - 0s 945us/sample - loss: 1.0299
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0221
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0114
Train on 500 samples
500/500 [==============================] - 0s 915us/sa

Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0293
Train on 500 samples
500/500 [==============================] - 0s 896us/sample - loss: 1.0215
Train on 500 samples
500/500 [==============================] - 0s 993us/sample - loss: 1.0108
Train on 500 samples
500/500 [==============================] - 0s 966us/sample - loss: 1.0264
Train on 500 samples
500/500 [==============================] - 0s 915us/sample - loss: 1.0141
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0257
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0148
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0168
Train on 500 samples
500/500 [==============================] - 0s 946us/sample - loss: 1.0295
Train on 500 samples
500/500 [==============================] - 0s 883us/sample - loss: 1.0264
Train on 500 samples
500/500 [============================

Train on 500 samples
500/500 [==============================] - 0s 805us/sample - loss: 1.0162
Train on 500 samples
500/500 [==============================] - 0s 831us/sample - loss: 1.0289
Train on 500 samples
500/500 [==============================] - 0s 858us/sample - loss: 1.0258
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0184
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0165
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0248
Train on 500 samples
500/500 [==============================] - 0s 938us/sample - loss: 1.0220
Train on 500 samples
500/500 [==============================] - 0s 838us/sample - loss: 1.0228
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0312
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0211
Train on 500 samples
500/500 [==============================

Train on 500 samples
500/500 [==============================] - 0s 844us/sample - loss: 1.0222
Train on 500 samples
500/500 [==============================] - 0s 859us/sample - loss: 1.0306
Train on 500 samples
500/500 [==============================] - 0s 911us/sample - loss: 1.0204
Train on 500 samples
500/500 [==============================] - 0s 873us/sample - loss: 1.0208
Train on 500 samples
500/500 [==============================] - 0s 840us/sample - loss: 1.0211
Train on 500 samples
500/500 [==============================] - 0s 801us/sample - loss: 1.0213
acc at 297 epoch is: 0.96325
Train on 500 samples
500/500 [==============================] - 0s 917us/sample - loss: 1.0036
Train on 500 samples
500/500 [==============================] - 0s 836us/sample - loss: 1.0187
Train on 500 samples
500/500 [==============================] - 0s 810us/sample - loss: 1.0416
Train on 500 samples
500/500 [==============================] - 0s 690us/sample - loss: 1.0016
Train on 500 samples


In [9]:
%memit

peak memory: 2418.77 MiB, increment: 0.42 MiB


In [10]:
print(maxnum,maxnmi,maxf1,maxprec)

0.96325 0.9057756412685753 0.96325 0.9633403532650222


In [4]:
import numpy as np
import time
x1=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/dmnist70kfromsenet.npy')
x1=x1.reshape(70000,2000)
x_test=x1[20000:,:]
y1=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/lmnist70kfromsenet.npy')
y1=y1.reshape(70000,)
y_test=y1[20000:,]

start=time.time()
encoder,encoding_dim,autoencoder=arch()
encoder.load_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/sdsc_mnist20k.ckpt')
ic=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/mnistpca2000/sdsc_mnist20k.npy')
z_test=encoder.predict(x_test)
assignment = assign_func(z_test, ic)
predictedlabels = np.argmax(assignment, axis=1)
rectified_label, acc=rectify_label(predictedlabels, y_test)
nmi=sklearn.metrics.normalized_mutual_info_score(y_test, predictedlabels)
f1=sklearn.metrics.f1_score(y_test, rectified_label,average='micro')
prec=sklearn.metrics.precision_score(y_test, rectified_label,average='weighted')
print(acc,nmi,f1,prec)
end=time.time()
print('time is',end-start)
%memit

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
0.96388 0.9067152589361864 0.96388 0.963943972238302
time is 11.677690982818604
peak memory: 1382.79 MiB, increment: 0.43 MiB
